In [1]:
#!/usr/local/bin/python3
# assign.py : Assign people to teams
#
# Code by: name IU ID
#
# Based on skeleton code by D. Crandall and B551 Staff, September 2021
#

In [2]:
import sys
import time
import numpy as np

# Cost function

In [3]:
def grading(groups):
    n_groups = len(groups)
    
    return n_groups*5

In [4]:
def wrong_group_size(group_assigned, want_group_size, groups_size):

    assigned_group_size=[groups_size[group] for group in group_assigned]
    group_size_compare = [assigned_group_size[n]==want_group_size[n] for n in range(len(group_assigned))]
    n_wrong_groups = len(people) - np.count_nonzero(group_size_compare)
    
    return n_wrong_groups*2


In [5]:
def wanted_not_in_assigned(people_assigned, want):

    people_assigned_mask = people_assigned.copy()
    for n, row in enumerate(want):
        for m, item in enumerate(row):
            if item == 'xxx':
                people_assigned_mask[n] = people_assigned_mask[n]+['xxx']

    match_wanted_assigned = [[want[i][n] in people_assigned_mask[i] for n in range(len(want[i]))] for i in range(len(people_assigned))]
    match_wanted_assigned = [elem for row in match_wanted_assigned for elem in row]
    
    wanted_not_assigned = len(match_wanted_assigned)- np.count_nonzero(match_wanted_assigned)
    
    return wanted_not_assigned *(0.05*60)

In [6]:
def not_wanted_in_assigned(people_assigned, not_want):    
    match_not_want_assigned = [[not_want[i][n] in people_assigned[i] for n in range(len(not_want[i]))] for i in range(len(people_assigned))]
    match_not_want_assigned = [elem for row in match_not_want_assigned for elem in row]
    not_want_assigned = np.count_nonzero(match_not_want_assigned)

    return not_want_assigned*10

# Check if the cost functions work on the example given in the instructions

In [24]:
filename = 'test1.txt'

with open(filename) as f:
    lines = f.readlines()

survey_text = [line.rstrip('\n') for line in lines]
survey_data = np.array([entry.split() for entry in survey_text])
survey_data

array([['djcran', 'djcran-vkvats-nthakurd', 'sahmaini'],
       ['sahmaini', 'sahmaini', '_'],
       ['sulagaop', 'sulagaop-xxx-xxx', '_'],
       ['fanjun', 'fanjun-xxx', 'nthakurd'],
       ['nthakurd', 'nthakurd', 'djcran,fanjun'],
       ['vkvats', 'vkvats-sahmaini', '_']], dtype='<U22')

In [25]:
#survey inputs
people = survey_data[:,0]
want = [item.split('-') for item in survey_data[:,1]]
not_want = [item.split(',') for item in survey_data[:,2]]
want_group_size = [len(item) for item in want]

print(f'people {people}')
print (f'want {want}')
print (f'not_want {not_want}')
print (f'want_group_size {want_group_size}')

people ['djcran' 'sahmaini' 'sulagaop' 'fanjun' 'nthakurd' 'vkvats']
want [['djcran', 'vkvats', 'nthakurd'], ['sahmaini'], ['sulagaop', 'xxx', 'xxx'], ['fanjun', 'xxx'], ['nthakurd'], ['vkvats', 'sahmaini']]
not_want [['sahmaini'], ['_'], ['_'], ['nthakurd'], ['djcran', 'fanjun'], ['_']]
want_group_size [3, 1, 3, 2, 1, 2]


In [32]:
# assign a group
#group example from assignment instructions
groups = [['djcran','vkvats','nthakurd'], ["sahmaini"], ["sulagaop", 'fanjun']]
print(f'groups from example {groups}')


groups from example [['djcran', 'vkvats', 'nthakurd'], ['sahmaini'], ['sulagaop', 'fanjun']]


In [33]:
# group stats
n_groups = len(groups)
groups_size = [len(group) for group in groups]

people_group_mat = [[people[x] in group for group in groups] for x in range(len(people))]
group_assigned = [people_group_mat[n].index(1) for n in range(len(people_group_mat))]

people_assigned =[groups[n] for n in group_assigned]

print (f'n_groups {n_groups}')
print(f'groups_size {groups_size}')
print (f'group_assigned {group_assigned}')
print (f'people_assigned {people_assigned}')

n_groups 3
groups_size [3, 1, 2]
group_assigned [0, 1, 2, 2, 0, 0]
people_assigned [['djcran', 'vkvats', 'nthakurd'], ['sahmaini'], ['sulagaop', 'fanjun'], ['sulagaop', 'fanjun'], ['djcran', 'vkvats', 'nthakurd'], ['djcran', 'vkvats', 'nthakurd']]


In [34]:
#calculate cost
cost_grading = grading(groups)
print(f'grading {cost_grading}')

cost_wrong_group_size = wrong_group_size(group_assigned, want_group_size, groups_size)
print(f'cost_wrong_group_size {cost_wrong_group_size}')

cost_wanted_not_in_group = wanted_not_in_assigned(people_assigned, want)
print(f'cost_wanted_not_in_group {cost_wanted_not_in_group}')

cost_not_wanted_in_group = not_wanted_in_assigned(people_assigned, not_want)
print(f'cost_not_wanted_in_group {cost_not_wanted_in_group}')

cost = cost_grading + cost_wrong_group_size + cost_wanted_not_in_group + cost_not_wanted_in_group
print (f'cost {cost}')

grading 15
cost_wrong_group_size 6
cost_wanted_not_in_group 3.0
cost_not_wanted_in_group 10
cost 34.0


# Random search - fixed number of iterations


In [35]:
def assign_random_groups(people):

    if len(people)% 3 ==0:
        poss_n_groups = int(len(people)/3)
    else:
        poss_n_groups = int(np.floor(len(people)/3))+1

    rand_groups = np.array([np.random.randint(0,poss_n_groups+1) for x in range(len(people))])

    n_in_group =[np.count_nonzero(rand_groups == n) for n in range(poss_n_groups+1)]

    while max(n_in_group)>3:
        rand_groups = np.array([np.random.randint(0,poss_n_groups) for x in range(len(people))])
        n_in_group =[np.count_nonzero(rand_groups == n) for n in range(poss_n_groups+1)]

    assigned_groups = []
    [assigned_groups.append(list(people[np.where(rand_groups ==n)])) for n in range(poss_n_groups+1)]

    groups = assigned_groups

    try:
        groups.pop([groups[n]==[]for n in range(len(groups))].index(1))
    except:
        pass

#     print(f'groups {groups}')
    
    return groups

In [68]:
def solver(filename):
    
#     filename ='test1.txt'    

    with open(filename) as f:
        lines = f.readlines()

    survey_text = [line.rstrip('\n') for line in lines]
    survey_data = np.array([entry.split() for entry in survey_text])

    #survey inputs
    people = survey_data[:,0]
    want = [item.split('-') for item in survey_data[:,1]]
    not_want = [item.split(',') for item in survey_data[:,2]]
    want_group_size = [len(item) for item in want]



    min_cost = 100000000
    n = 1
    runs = 10000
    for n in range(runs):
        #randomly generated group
        rand_groups = assign_random_groups(people)
        groups = rand_groups
#         group example from assignment instructions
#         groups = [['djcran','vkvats','nthakurd'], ["sahmaini"], ["sulagaop", 'fanjun']]
#         print(f'groups from example {groups}')

        # group stats
        n_groups = len(groups)
        groups_size = [len(group) for group in groups]

        people_group_mat = [[people[x] in group for group in groups] for x in range(len(people))]
        group_assigned = [people_group_mat[n].index(1) for n in range(len(people_group_mat))]

        people_assigned =[groups[n] for n in group_assigned]


        #calculate cost
        cost_grading = grading(groups)
        cost_wrong_group_size = wrong_group_size(group_assigned, want_group_size, groups_size)
        cost_wanted_not_in_group = wanted_not_in_assigned(people_assigned, want)
        cost_not_wanted_in_group = not_wanted_in_assigned(people_assigned, not_want)

        cost = cost_grading + cost_wrong_group_size + cost_wanted_not_in_group + cost_not_wanted_in_group


        if cost < min_cost:
            min_cost_groups = groups
            min_cost = cost
            print(n, min_cost_groups, min_cost)
        
        n+=1
        
    min_cost_groups_str = ['-'.join(x) for x in min_cost_groups]
    print (f'\nmin_cost_group after {runs} random samples is {min_cost_groups_str, min_cost}')

    result = {"assigned-groups": min_cost_groups_str,
              "total-cost" : min_cost}

    

    
    
    return result


In [69]:
solver('test3.txt')

0 [['nthakurd', 'wang5', 'zhang'], ['sahmaini', 'sulagaop', 'wu'], ['sun', 'li', 'zheng'], ['djcran', 'wang', 'feng'], ['vkvats', 'li4'], ['fanjun', 'qian', 'chen'], ['zhao', 'zhou', 'zhang3']] 116.0
2 [['zhao', 'li4', 'zhang'], ['sahmaini', 'zhang3'], ['fanjun', 'sun', 'zhou'], ['qian', 'wu', 'chen'], ['sulagaop', 'nthakurd', 'feng'], ['djcran', 'wang', 'wang5'], ['vkvats', 'li', 'zheng']] 111.0
4 [['qian', 'wu'], ['sahmaini', 'fanjun', 'zhao'], ['djcran', 'sulagaop', 'wang5'], ['vkvats', 'zhou', 'zhang'], ['sun', 'li', 'feng'], ['zheng', 'wang', 'zhang3'], ['nthakurd', 'chen', 'li4']] 106.0
40 [['sulagaop', 'vkvats', 'zhao'], ['nthakurd', 'sun', 'feng'], ['li', 'zhou'], ['djcran', 'fanjun', 'wang5'], ['qian', 'chen', 'zhang'], ['wu', 'zhang3', 'li4'], ['sahmaini', 'zheng', 'wang']] 103.0
46 [['sahmaini', 'fanjun', 'zhang3'], ['nthakurd', 'wang', 'li4'], ['qian', 'zhou', 'zheng'], ['sulagaop', 'zhao', 'feng'], ['sun', 'li', 'wang5'], ['djcran', 'wu', 'zhang'], ['vkvats', 'chen']] 102.

{'assigned-groups': ['djcran-chen-zhang3',
  'wu-zheng-wang',
  'sahmaini-nthakurd-zhou',
  'fanjun-vkvats',
  'sulagaop-li4-zhang',
  'zhao-feng-wang5',
  'qian-sun-li'],
 'total-cost': 93.0}

# Random Search - Yield when a lower cost is found

In [67]:
def solver_yield(filename):
    
#     filename ='test1.txt'    

    with open(filename) as f:
        lines = f.readlines()

    survey_text = [line.rstrip('\n') for line in lines]
    survey_data = np.array([entry.split() for entry in survey_text])

    #survey inputs
    people = survey_data[:,0]
    want = [item.split('-') for item in survey_data[:,1]]
    not_want = [item.split(',') for item in survey_data[:,2]]
    want_group_size = [len(item) for item in want]



    min_cost = 100000000
    while min_cost>0:
        #randomly generated group
        rand_groups = assign_random_groups(people)
        groups = rand_groups
#         group example from assignment instructions
#         groups = [['djcran','vkvats','nthakurd'], ["sahmaini"], ["sulagaop", 'fanjun']]
#         print(f'groups from example {groups}')

        # group stats
        n_groups = len(groups)
        groups_size = [len(group) for group in groups]

        people_group_mat = [[people[x] in group for group in groups] for x in range(len(people))]
        group_assigned = [people_group_mat[n].index(1) for n in range(len(people_group_mat))]

        people_assigned =[groups[n] for n in group_assigned]


        #calculate cost
        cost_grading = grading(groups)
        cost_wrong_group_size = wrong_group_size(group_assigned, want_group_size, groups_size)
        cost_wanted_not_in_group = wanted_not_in_assigned(people_assigned, want)
        cost_not_wanted_in_group = not_wanted_in_assigned(people_assigned, not_want)

        cost = cost_grading + cost_wrong_group_size + cost_wanted_not_in_group + cost_not_wanted_in_group


        if cost < min_cost:
            min_cost_groups = groups
            min_cost = cost
        
        
            min_cost_groups_str = ['-'.join(x) for x in min_cost_groups]

            result = {"assigned-groups": min_cost_groups_str,
                      "total-cost" : min_cost}

    

    
    
            yield (result)
        



In [66]:
for result in solver_yield('test3.txt'):
    print("----- Latest solution:\n" + "\n".join(result["assigned-groups"]))
    print("\nAssignment cost: %d \n" % result["total-cost"])

----- Latest solution:
sulagaop-qian-wu
li-zhou-chen
zheng-feng
sahmaini-nthakurd-wang5
fanjun-li4-zhang
djcran-zhao-zhang3
vkvats-sun-wang

Assignment cost: 112 

----- Latest solution:
djcran-qian-zhou
vkvats-zhao-zheng
sulagaop-chen-wang5
li-feng-zhang
sahmaini-wang-zhang3
fanjun-sun-li4
nthakurd-wu

Assignment cost: 110 

----- Latest solution:
djcran-wang-zhang
chen-wang5
fanjun-vkvats-zhou
sun-li-feng
zhao-wu-zhang3
sahmaini-sulagaop-li4
nthakurd-qian-zheng

Assignment cost: 109 

----- Latest solution:
zhang3-wang5
qian-sun-li
djcran-wu-li4
sulagaop-wang-chen
fanjun-zhao-zheng
sahmaini-nthakurd-zhang
vkvats-zhou-feng

Assignment cost: 107 

----- Latest solution:
fanjun-zhang3-zhang
sulagaop-nthakurd-li4
djcran-vkvats-qian
zhao-feng-chen
li-zhou-zheng
sahmaini-sun-wang
wu-wang5

Assignment cost: 103 

----- Latest solution:
fanjun-zhou
vkvats-sun-li
sulagaop-zhao-wang5
nthakurd-zhang3-li4
sahmaini-qian-zhang
zheng-feng-chen
djcran-wu-wang

Assignment cost: 102 

----- Latest sol

KeyboardInterrupt: 

In [ ]:
def solver(input_file):
    """
    1. This function should take the name of a .txt input file in the format indicated in the assignment.
    2. It should return a dictionary with the following keys:
        - "assigned-groups" : a list of groups assigned by the program, each consisting of usernames separated by hyphens
        - "total-cost" : total cost (time spent by instructors in minutes) in the group assignment
    3. Do not add any extra parameters to the solver() function, or it will break our grading and testing code.
    4. Please do not use any global variables, as it may cause the testing code to fail.
    5. To handle the fact that some problems may take longer than others, and you don't know ahead of time how
       much time it will take to find the best solution, you can compute a series of solutions and then
       call "yield" to return that preliminary solution. Your program can continue yielding multiple times;
       our test program will take the last answer you 'yielded' once time expired.
    """

    # Simple example. First we yield a quick solution
    yield({"assigned-groups": ["vibvats-djcran-zkachwal", "shah12", "vrmath"],
               "total-cost" : 12})

    # Then we think a while and return another solution:
    time.sleep(10)
    yield({"assigned-groups": ["vibvats-djcran-zkachwal", "shah12-vrmath"],
               "total-cost" : 10})

    # This solution will never befound, but that's ok; program will be killed eventually by the
    #  test script.
    while True:
        pass
    
    yield({"assigned-groups": ["vibvats-djcran", "zkachwal-shah12-vrmath"],
               "total-cost" : 9})

In [ ]:
if __name__ == "__main__":
    if(len('XX') != 2):
        raise(Exception("Error: expected an input filename"))

    for result in solver('test1.txt'):
        print("----- Latest solution:\n" + "\n".join(result["assigned-groups"]))
        print("\nAssignment cost: %d \n" % result["total-cost"])
    

[['qian', 'wang5'], ['vkvats', 'zhao', 'feng'], ['zhou', 'wang', 'zhang3'], ['sulagaop', 'nthakurd', 'chen'], ['sahmaini', 'fanjun', 'zheng'], ['sun', 'li', 'wu'], ['djcran', 'li4', 'zhang']] 146


TypeError: sequence item 0: expected str instance, list found